Link to follow for this : https://www.mlexpert.io/prompt-engineering/chatbot-with-local-llm-using-langchain#try-the-**model**

#Install and load packages

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.40.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.30.0 --progress-bar off
!pip install -qqq accelerate==0.21.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq langchain==0.0.233 --progress-bar off

###Hugging face login

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

#Load model and start server

###Imports

In [ ]:
import re
import warnings
from typing import List

import torch
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseOutputParser
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    pipeline,
)

warnings.filterwarnings("ignore", category=UserWarning)

!pip install peft
import bitsandbytes as bnb
from transformers import BitsAndBytesConfig
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

  Obtaining dependency information for peft from https://files.pythonhosted.org/packages/88/a0/6e1c23293a922a9c9e9bd8d56a60cd78ecf531fdabe45ac975e142bfbe86/peft-0.4.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.7 MB/s eta 0:00:00


### Load fine tuned model with qLoRA weights in 4 bit config

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


PEFT_MODEL = "arvind2626/f6"
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


###Generation config used by the model

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 900
generation_config.temperature = 0
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

###Function to stop generation on specific tags

In [ ]:
class StopGenerationCriteria(StoppingCriteria):
    def __init__(
        self, tokens: List[List[str]], tokenizer: AutoTokenizer, device: torch.device
    ):
        stop_token_ids = [tokenizer.convert_tokens_to_ids(t) for t in tokens]
        self.stop_token_ids = [
            torch.tensor(x, dtype=torch.long, device=device) for x in stop_token_ids
        ]

    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
    ) -> bool:
        for stop_ids in self.stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids) :], stop_ids).all():
                return True
        return False

In [ ]:
stop_tokens = [["Human", ":"], ["AI", ":"]]
stopping_criteria = StoppingCriteriaList(
    [StopGenerationCriteria(stop_tokens, tokenizer, model.device)]
)

###Initialize generation pipeline

In [ ]:
generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task="text-generation",
    stopping_criteria=stopping_criteria,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=generation_pipeline)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerN

###Clean the final output

In [ ]:
class CleanupOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        user_pattern = r"\nUser"
        text = re.sub(user_pattern, "", text)
        human_pattern = r"\nHuman:"
        text = re.sub(human_pattern, "", text)
        ai_pattern = r"\AI:"
        return re.sub(ai_pattern, "", text).strip()

    @property
    def _type(self) -> str:
        return "output_parser"

###Template given to AI chatbot

In [ ]:
purchase_history = "(Blue Denim Jacket, White Graphic T-Shirt, Red Plaid Flannel Shirt, Gray Hooded Sweatshirt, Navy Blue Chinos, Striped Polo Shirt, Brown Leather Belt, Floral Print Sundress)"
# Trends Generated from Trend scraper
trends = "(Dungarees, Oversized Denim Jackets, Cutout Bodysuit, Cutout Tops, Lehenga Skirts, Cropped Leather Jackets, Velvet Blazers with Rich Colors, Cropped Leather Jackets, Lehenga Skirts, Floor-Length Anarkali, Skirt with Side Slit, Cropped Jumpsuit)"

In [ ]:
template = """
System: This is a conversation between a human and an outfit recommendation AI. The AI does a conversation with the user to get to know their various details such as age, height, location, ocassion, preferences. Based on these details it suggests an outfit to the human.

This is a list of past purchases of the human
"""+purchase_history+"""\n

This is a list of trending outfits
"""+trends+"""\n
The AI considers the human's purchase history and current trending outfits and can suggest outfits which might be similar. Outfits need not necessarily belong to these.


The AI must always suggest 3 outfits in collections of topwear, bottomwear, shoes, accesories in the following format:
Outfit 1:
Top: Name of cloth: short description
Bottom: Name of cloth: short description
Footwear:...
Accesories:...

Outfit 2:
Top: ...
Bottom: ...
and so on

The AI does not suggest brands or price. It only suggest different types of clothes

The AI doesn't immediately suggest an outfit. It does a conversation/chat with the human for getting various details. It suggests the outfit only when the Human asks so. It suggests outfit strictly in the pattern mentioned above.


Current conversation:

{history}

Human: {input}
AI:
""".strip()

prompt = PromptTemplate(input_variables=["history", "input"], template=template)

###Start the Langchain, Conversation Beffer Window Memory

In [ ]:
memory = ConversationBufferWindowMemory(
    memory_key="history", k=6, return_only_outputs=True
)

chain = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    output_parser=CleanupOutputParser(),
    verbose=True,
)

##API endpoint

In [ ]:
pip install fastapi
pip install colabcode

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
@app.post("/prompt")
def create(item: Item):
  dt = item.dict()
  text = dt["text"]
  res = chain(text)
  print(res["response"])
  return res["response"]

Hi, I'm a 19 year old female college student from Pune.


> Entering new ConversationChain chain...
Prompt after formatting:

System: This is a conversation between a human and an outfit recommendation AI. The AI does a conversation with the user to get to know their various details such as age, height, location, ocassion, preferences. Based on these details it suggests an outfit to the human.

This is a list of past purchases of the human
(Blue Denim Jacket, White Graphic T-Shirt, Red Plaid Flannel Shirt, Gray Hooded Sweatshirt, Navy Blue Chinos, Striped Polo Shirt, Brown Leather Belt, Floral Print Sundress)


This is a list of trending outfits
(Dungarees, Oversized Denim Jackets, Cutout Bodysuit, Cutout Tops, Lehenga Skirts, Cropped Leather Jackets, Velvet Blazers with Rich Colors, Cropped Leather Jackets, Lehenga Skirts, Floor-Length Anarkali, Skirt with Side Slit, Cropped Jumpsuit)The AI considers the human's purchase history and current trending outfits and can suggest outfits whi

KeyboardInterrupt: ignored

In [ ]:
from colabcode import ColabCode
ColabCode()